In [1]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from datetime import datetime 

import time

In [2]:
circle="Edc Gonda"
month="jul2023"
month_doc=5
month_unpaid=7

In [3]:
def loop(file_name):
#     encoding = "ISO-8859-1"
    return pd.read_csv(f"{month}/{circle}/rural/{file_name}", encoding = "ISO-8859-1")

In [4]:
def NonGOVT(df):
    govt = [30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,48,72,73,74,75,80,81,82,101,102,103,104,105,106,107,'80B','42A','41A','40A','H31'
           'H32','H33','H41','H42','H43','HV41','H13']
    govt = [str(x) for x in govt]
    df['gov_flag'] = df['SUPPLY_TYPE'].isin(govt)
    return df

In [5]:
def billable(df):
    bil = ['Live','First Bill Issued','New Connection','Temp.Disconnected']
    df['bilable_flag'] = df['BILLING_STATUS'].isin(bil)
    return df

In [6]:
def arrear(df):
#     df = df[df['TOTPAYAMT'] > 0]
    df['ARREAR_FILTER'] = None
    df.loc[df['AMOUNT'] < 5000,"ARREAR_FILTER"] = "<5k"
    df.loc[((df['AMOUNT'] >= 5000) &(df['AMOUNT'] < 10000) ),"ARREAR_FILTER"]= ">=5k-10k"
#     df.loc[df['TOTPAYAMT'] <= 10000,"ARREAR_FILTER"] = "<=10k"
    df.loc[((df['AMOUNT'] >= 10000) & (df['AMOUNT'] < 100000)),"ARREAR_FILTER"] = "10k-1L"
#     df.loc[((df['AMOUNT'] >= 10000) & (df['AMOUNT'] < 50000)),"ARREAR_FILTER"] = "10k-50k"
    df.loc[(df['AMOUNT'] >= 100000),"ARREAR_FILTER"] = ">=1L"
#     df.loc[df['AMOUNT'] >= 50000,"ARREAR_FILTER"] = ">=50k"
    
    return df
    

In [7]:
def CatLoad(df):
    df['CONTRACTLOAD'] = pd.to_numeric(df['CONTRACTLOAD'],errors="coerce")
    df['LOAD_FILTER'] = None
    df.loc[df['CONTRACTLOAD'] < 5,"LOAD_FILTER"] = "0-5kw"
    df.loc[((df['CONTRACTLOAD'] >= 5) & (df['CONTRACTLOAD'] < 10)),"LOAD_FILTER"] = "5-9kw"
    df.loc[df['CONTRACTLOAD'] >= 10,"LOAD_FILTER"] = "great than 10kw"
    return df

    

In [8]:
def relGov(df):
#     ((df['gov_flag'] == False) & ( (pd.isnull(df['GOVT_FLAG'] )) | (df['GOVT_FLAG'] == 'NON-GOVERNMENT'))) 
    df['relGov'] = 'Goverment'
    df.loc[((df['gov_flag'] == False) & ( (pd.isnull(df['GOVT_FLAG'] )) | (df['GOVT_FLAG'] == 'NON-GOVERNMENT'))) ,"relGov"] = 'Non-Goverment'
    return df

In [9]:
def faulty_Ageing(df):
    df['METER_FAULT_CNT'] = pd.to_numeric(df['METER_FAULT_CNT'],errors="coerce")
    df['faulty_Ageing'] = None
    df.loc[df['METER_FAULT_CNT'] < 2,"faulty_Ageing"] = "faulty<2"
    df.loc[((df['METER_FAULT_CNT'] >= 2) & (df['METER_FAULT_CNT'] < 3)),"faulty_Ageing"] = "faulty>=2 TO <3"
    df.loc[df['METER_FAULT_CNT'] >= 3,'METER_FAULT_CNT'] = "faulty >=3"
    return df

In [10]:
def date(df):
    df['LAST_PAID_DATE'] = pd.to_datetime(df['LAST_PAID_DATE'],format='%d-%m-%Y',errors = "coerce")
    df['SUPP_RELEASE_DATE'] = pd.to_datetime(df['SUPP_RELEASE_DATE'],format='%d-%b-%Y',errors = "coerce")
#     df['BILLDATE'] = pd.to_datetime(df['BILLDATE'],format="%d-%m-%Y %H:%M:%S",errors = "coerce")
#     df['LAST_MU_RDG_DATE'] = pd.to_datetime(df['LAST_MU_RDG_DATE'],format="%d-%b-%y",errors = "coerce")
    df['KYC_CREATE_DATE'] = pd.to_datetime(df['KYC_CREATE_DATE'],format="%d-%b-%y",errors = "coerce")
#     df['DISCONNECTION_DT'] = pd.to_datetime(df['DISCONNECTION_DT'],format='%d-%b-%y',errors = "coerce")
#     df['DISCONNECTION_CREATE_DT'] = pd.to_datetime(df['DISCONNECTION_CREATE_DT'],format='%d-%b-%y',errors = "coerce")
    
    return df

In [11]:
def unpaid(df):
    df['unpaid_status'] = None
    df.loc[df['LAST_PAID_DATE'] < pd.Timestamp(2023, 1, month_unpaid),"unpaid_status"] = "unpaid"
    df.loc[(pd.isnull(df['LAST_PAID_DATE'] )),"unpaid_status"] = "unpaid"     
    df.loc[df['LAST_PAID_DATE'] >= pd.Timestamp(2023, 1, month_unpaid),"unpaid_status"] = "paid"
    
    return df

In [12]:
def Neverpaid(df):
    df['Neverpaid_status'] = None
    df.loc[((pd.isnull(df['LAST_PAID_DATE'] )) & (df['SUPP_RELEASE_DATE'] < pd.Timestamp(2023, 1, month_doc))),"Neverpaid_status"] = 1
    df.loc[pd.isnull(df['Neverpaid_status']),"Neverpaid_status"] = 0
    
    return df

In [13]:
def kyc_status(df):
    df['kyc_status'] = None
    df.loc[df['KYC_CREATE_DATE'] >= pd.Timestamp(2022, 1, 9),"kyc_status"] = 1
    df.loc[pd.isnull(df['KYC_CREATE_DATE']) ,"kyc_status"] = 0     #df[~pd.isnull(df['Courses'])]
    df.loc[df['KYC_CREATE_DATE'] < pd.Timestamp(2022, 1, 9),"kyc_status"] = 0
    
    return df

In [14]:
def cat_date(df):
    df['cat_date'] = None
    

In [15]:
import os

mylist = os.listdir(f"{month}/{circle}/rural")

In [16]:
mylist

['GONDA_ZONE__0723_45.CSV',
 'GONDA_ZONE__0723_CG.CSV',
 'GONDA_ZONE__0723_GM.CSV',
 'MD_REPORT_0723_GD.CSV']

In [17]:
start = time.time()
df = []
df = Parallel(n_jobs=-1, verbose=10,prefer="threads")(delayed(loop)(file_name) for file_name in mylist)
# print(df)
df = pd.concat(df, ignore_index=True)

end = time.time()
print("CSV:", end - start)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
C:\Users\pc\AppData\Local\Temp\ipykernel_3324\4049885345.py:3: DtypeWarning: Columns (0,7,174,186,219) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f"{month}/{circle}/rural/{file_name}", encoding = "ISO-8859-1")
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    4.4s
C:\Users\pc\AppData\Local\Temp\ipykernel_3324\4049885345.py:3: DtypeWarning: Columns (184,211,219) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f"{month}/{circle}/rural/{file_name}", encoding = "ISO-8859-1")
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:   12.0s remaining:   12.0s
C:\Users\pc\AppData\Local\Temp\ipykernel_3324\4049885345.py:3: DtypeWarning: Columns (0,186,194) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f"{month}/{circle}/rural/{file_name}", encoding = "ISO-8859-1")


CSV: 16.776853561401367


In [18]:
df.dtypes

BOOK_CODE             int64
DIVISION             object
DIVISION_ID          object
SUBDIVISION          object
SUBSTATION           object
                     ...   
WHATSAPP_NO         float64
INOPERATIVE_AMT     float64
INOPERATIVE_DATE     object
KYC_CREATE_DATE      object
HV_BILL_FLAG         object
Length: 222, dtype: object

In [19]:
df.DIVISION.unique()

array(['EDD-I-BAHRAICH', 'EDD-III-BAHRAICH-KAISERGANJ',
       'EDD-II-BAHRAICH-NANPARA'], dtype=object)

In [20]:
column =list(df.columns)

In [51]:
df['BILLING_STATUS'].unique()

array(['Live', 'Account Closed', 'Temp.Disconnected', 'First Bill Issued',
       'Bill Stopped', 'Dismantled', 'New Connection'], dtype=object)

In [18]:
column=['BOOK_CODE',
 'DIVISION',
 'DIVISION_ID',
 'SUBDIVISION',
 'SUBSTATION',
 'FEEDER',
 'SUPPLY_TYPE',
 'SCNO',
 'OLDSCNO',
 'BILLDATE',
 'BILLNO',
 'BILLED_MONTHS',
 'BILLDUEDATE',
 'BILLDISCON',
 'ADDRESS1',
 'ADDRESS2',
 'ADDRESS3',
 'NAME',
'FATHER_NAME',
        'VILLAGE_NAME',
         'MTR_NUMBER',
 'OWNERSHIP',
 'PIN',
 'PHONE',
 'EMAIL',
 'BILLBASIS',
 'CONTRACTLOAD',
 'VOLSUPLY',
 'PF',

 'METERED_KWH',
 'METERED_KVAH',
 'ADJ_UNITS',
 'BILLABLE_UNIT',
 'MAX_DEMAND',
 'ENRGYCHG',
 'MCG',
 'FIXEDCHG',
 'DEMANDCHG',
 'EXCESSDCHG',
 'EDCHG',
 'VOLSUPREB',
 'VOLSUPSURCHG',
 'CAPACITOR',
 'LPFSURCHARGE',
 'SOLARCAP',
 'SOLARREBATE',
 'PROVADJ',
 'TOTBILL',
 'SUBSIDY',
 'CURTOTAMT',
 'LATEPAYSUR',
 'OLDDUEAMT',
 'TOTPAYAMT',
 'AMOUNT',
 'AMTAFTRDUE',
 'AMTAFTRDIS',
 'CONSID',
 'BILLING_STATUS',
 'SUPP_RELEASE_DATE',
 'MF',
 'BILLID',
 'PREV_RDG_DATE',
 'PRST_RDG_DATE',
 'PREV_RDG_KWH',
 'PREV_RDG_KVAH',
 'PRST_RDG_KWH',
 'PRST_RDG_KVAH',
 'BILLED_CATEGORY',
 'APPROVE',
 'GRACEDATE',
 'ADDL_SURCHARGE',
 
 'CAT_CODE',
 'MTR_TYPE',
 'BILL_GRP',
 'SECURITY',
 'CATID',
 'ICDAMT',
 'PROVAMT',
 'PROVPERIOD',
 'PROVUNITS',
 'ARREAR',
 'CIN',
 'LOAD_UNIT',
 'BILL_CYCLE',
 'ARREAR_SURCHG',
 'ASSESED_UNITS',
 'CHEQUEBOUNCE_DATE',
 'ISCHK_RESTRICTION',
 'BLISSUEDATE',
 'BILLED_KWH',
 'MTR_STATUS_CODE',
 'CUML_ACTUAL_UNITS',
 'CUML_BILL_UNITS',
 'BILLING_FLAG',
 'TOTPAYMENTADJ',
 'ROUNDAMT',
 'OTHER_DUES',
 'MTR_FLG',
 'PREV_BLISSUE_DT',
 'PREV_FIN_YR_ARR',
 'PREV_FIN_YR_LPS',
 'CURR_FIN_YR_ARR',
 'CURR_FIN_YR_LPS',
 'ARR_COUNT',
 'BILLDISCON1',
 'CREATE_DATE',
 'MCG_EC_ADJ',
 'MCG_ADJ_CF',
 'FCA_ADJ_AMT',
 'FSA_CHG',
 'CUST_ADJ_AMT',
 'BILL_TYPE',
 'ACCOUNTNO',
 'MTR_STATUS',
 'ICD_AMT',
 'MINIMUM_CHG',
 'REG_SURCHG1',
 'REG_SURCHG2',
 'REBATE_GIVEN',
 'BILL_FROM_DT',
 'BILL_TO_DT',
 'ADDITIONAL_REBATE',
 'POWER_LOOM_CHG',
 'POWER_LOOM_ED',
 'NO_OF_LOOMS_LS60',
 'NO_OF_LOOMS_GT60',
 'DISCOUNT',
 'DEBIT',
 'CREDIT',
 'ARR_ADLEC_CHG',
 'ARR_ADLED_CHG',
 'PRVYRARR',
 'PRVYRLPS',
 'LAST_PAID_AMT',
 'LAST_PAID_DATE',
 'IMEI_NO',
 'LATITUDE',
 'LONGITUDE',
 'GOVT_FLAG',
 'PHASE',
 'METER_FAULT_CNT',
 'MD_KW',
 'MD_KVA',
 'CONSUMPTION_UNIT',
 'PROV_BILLED_KWH',
 'PROV_BILLED_KVAH',
 'AMOUNT_PAID_IN_BILLMONTH',
 'AMOUNTPAID_CNT_IN_BILLMONTH',
 'AMOUNT_PAID_REPORT_MONTH',
 'AMOUNTPAID_CNT_REPORT_MONTH',
 'MTR_MAKE',
 'OFFICE_ID',
 'REPORT_MONTH',
 'REPORT_YEAR',
 'FEEDER_ACODE',
 'DISTRICT_NAME',

 'VILLAGE_CODE',
 'HABITATION_NAME',
 'HABITATION_CODE',
 'LAST_OK_RDG_KWH',
 'LAST_OK_RDG_KVAH',
 'LAST_MU_RDG_DATE',
 'LAST_3_MONTHS_CONSUMPTION',
 'READER_LOGIN_ID',
 'SCHEME',
 'SCHEME_TYPE',
 'SOURCE_TYPE',
 'PR_TOWN',
 'CURYRARR',
 'CURYRLPS',
 'MTR_OPNRDGDT',
 'MTR_OPNRDG',
 'METER_STATUS',
 'MDI',
 'METER_READ_REMARK',
 'MR_SOURCE_CD',
 'CONSUMER_CREATE_DT',
 'MTR_RDR_OBSERVATION',
 'MT_CHANGE_DATE',
 'ST_CHANGE_DATE',
 'OLD_ST',
 'ED_AVAIL_ADJ',
 'ED_CUR_ADJ',
 'SERVICEREGION',
 'UNBILLED_EXCEPTION',
 'LAST_ACD_AMT',
 'LAST_ACD_COL_DT',
 'OLD_BR_AMOUNT',
 'DISCONNECTION_DT',
 'DISCONNECTION_CREATE_DT',
 'SMID',
 'FMID',
 'READER_NAME',
 'READER_MOBILE_NO',
 'INSTALLMENT_FLAG',
 'NO_OF_PENDING_INS',
 'INST_AMOUNT',
 'ACTUAL_MTR_NUMBER',
 'ACTUAL_MTR_MAKE_NAME',
 'GOVT_CODE',
 'GOVT_TYPE',
 'RECONN_CHG',
 'PROBE_OBSERVATION',
 'TCS_CHG',
 'CONSUMER_TYPE',
 'PANNUM',
 'OPR_FLAG',
 'SDO_CODE',
 'FEEDER_CODE',
 'DTR_CODE',
 'DTR_NAME',
 'DT_GISUID',
 'ROUTE_NAME',
 'PAYMENT_TYPE',
 'PLOOM_CHG',
 'PTW_REBATE',
 'PTW50REBATE',
 'WHATSAPP_NO',
 'INOPERATIVE_AMT',
 'INOPERATIVE_DATE',
 'KYC_CREATE_DATE',
 'HV_BILL_FLAG']

In [22]:
im_col = ['DIVISION',
          'SCNO',
          'ACCOUNTNO',

 'SUBDIVISION',
 'SUBSTATION',
 'FEEDER',
 'SUPPLY_TYPE',
          'BILLING_STATUS',
          'CAT_CODE',
          'ADDRESS1',
          'VILLAGE_NAME',
 'VILLAGE_CODE',
 'HABITATION_NAME',
 'HABITATION_CODE',
          
 'ADDRESS2',
 'ADDRESS3',
 'NAME','FATHER_NAME','PHONE','MTR_NUMBER', 'BILLBASIS','BILLDATE','METERED_KWH','BILLABLE_UNIT',
        
 'CONTRACTLOAD','TOTBILL','CURTOTAMT','TOTPAYAMT',
 'AMOUNT','CURYRARR',
 'CURYRLPS','SUPP_RELEASE_DATE','BILLED_CATEGORY','MTR_STATUS','LAST_PAID_AMT',
 'LAST_PAID_DATE','LAST_OK_RDG_KWH','METER_FAULT_CNT',
          'MT_CHANGE_DATE',
 'ST_CHANGE_DATE','LAST_MU_RDG_DATE',
        'READER_NAME',
 'READER_MOBILE_NO',
 'LAST_OK_RDG_KVAH',
 'LAST_MU_RDG_DATE',
 'GOVT_FLAG','GOVT_CODE',
          
 'GOVT_TYPE','PAYMENT_TYPE','AMOUNT_PAID_IN_BILLMONTH','SCHEME','OPR_FLAG','KYC_CREATE_DATE','DISCONNECTION_DT','DISCONNECTION_CREATE_DT','AMOUNT_PAID_IN_BILLMONTH']

In [19]:
df = df[column]

In [20]:
df_duplicate=df[df.duplicated(subset=['NAME','FATHER_NAME','VILLAGE_NAME'],keep=False)]

In [21]:
df_duplicate['duplicate'] = False
df_duplicate['duplicate']=df.duplicated(subset=['NAME','FATHER_NAME','VILLAGE_NAME'])

C:\Users\pc\AppData\Local\Temp\ipykernel_3324\2424249031.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_duplicate['duplicate'] = False
C:\Users\pc\AppData\Local\Temp\ipykernel_3324\2424249031.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_duplicate['duplicate']=df.duplicated(subset=['NAME','FATHER_NAME','VILLAGE_NAME'])


In [55]:
df['BILLING_STATUS'].unique()

array(['Live', 'Account Closed', 'Temp.Disconnected', 'First Bill Issued',
       'Bill Stopped', 'Dismantled', 'New Connection'], dtype=object)

In [56]:
df['srd']=df['SUPP_RELEASE_DATE'].copy()

In [57]:
df['KYC_CREATE_DATE'].unique()

array(['16-MAY-23', '09-JUN-23', nan, '17-MAY-23', '30-MAY-23',
       '12-MAY-23', '03-JUN-23', '13-MAY-23', '01-JUL-23', '31-MAY-23',
       '09-MAY-23', '02-JUN-23', '26-FEB-23', '29-JUN-23', '04-SEP-22',
       '14-MAY-23', '27-JUN-23', '11-MAY-23', '20-MAY-23', '07-JUN-23',
       '18-JUL-23', '15-MAY-23', '19-MAY-23', '29-MAY-23', '30-JUN-23',
       '18-MAY-23', '08-JUN-23', '13-APR-23', '22-JUL-23', '14-FEB-23',
       '11-FEB-23', '15-FEB-23', '03-FEB-23', '26-JUN-23', '06-SEP-22',
       '10-SEP-22', '28-MAY-23', '28-JAN-23', '05-JUN-23', '19-NOV-22',
       '02-FEB-23', '26-MAY-23', '27-MAY-23', '08-OCT-22', '11-NOV-22',
       '13-SEP-22', '16-JUN-23', '22-MAY-23', '27-MAR-23', '21-FEB-23',
       '27-FEB-23', '19-JUL-23', '01-JUN-23', '06-JUN-23', '09-MAR-23',
       '12-JUL-23', '07-SEP-22', '23-JUN-23', '28-MAR-23', '28-JUN-23',
       '16-NOV-22', '30-JAN-23', '12-JAN-23', '08-MAY-23', '04-FEB-23',
       '05-APR-23', '13-FEB-23', '17-JUL-23', '10-MAY-23', '05-SEP-22',


In [58]:
df['lpd']=df['LAST_PAID_DATE'].copy()

In [59]:
df['BILLING_STATUS'].unique()

array(['Live', 'Account Closed', 'Temp.Disconnected', 'First Bill Issued',
       'Bill Stopped', 'Dismantled', 'New Connection'], dtype=object)

In [60]:
pd.to_datetime(df['DISCONNECTION_DT'][:5],format="%d-%b-%y")

0   NaT
1   NaT
2   NaT
3   NaT
4   NaT
Name: DISCONNECTION_DT, dtype: datetime64[ns]

In [61]:
df['BILLING_STATUS'].unique()

array(['Live', 'Account Closed', 'Temp.Disconnected', 'First Bill Issued',
       'Bill Stopped', 'Dismantled', 'New Connection'], dtype=object)

In [62]:
df = date(df)
df = unpaid(df)
df = Neverpaid(df)
df=kyc_status(df)

In [63]:
df['KYC_CREATE_DATE'].unique()

<DatetimeArray>
['2023-05-16 00:00:00', '2023-06-09 00:00:00',                 'NaT',
 '2023-05-17 00:00:00', '2023-05-30 00:00:00', '2023-05-12 00:00:00',
 '2023-06-03 00:00:00', '2023-05-13 00:00:00', '2023-07-01 00:00:00',
 '2023-05-31 00:00:00',
 ...
 '2022-10-13 00:00:00', '2022-10-15 00:00:00', '2022-10-14 00:00:00',
 '2022-11-01 00:00:00', '2022-12-03 00:00:00', '2022-10-06 00:00:00',
 '2023-01-06 00:00:00', '2022-09-17 00:00:00', '2022-10-16 00:00:00',
 '2023-01-01 00:00:00']
Length: 341, dtype: datetime64[ns]

In [64]:
df['LAST_PAID_DATE']

0               NaT
1               NaT
2               NaT
3               NaT
4        2023-07-17
            ...    
466948   2023-06-08
466949   2022-05-05
466950   2020-09-14
466951   2023-04-19
466952          NaT
Name: LAST_PAID_DATE, Length: 466953, dtype: datetime64[ns]

In [65]:
# df['LAST_PAID_DATE'].isna()

In [66]:
df['SUPP_RELEASE_DATE'].unique()

<DatetimeArray>
['2019-06-06 00:00:00', '1999-01-25 00:00:00', '2018-08-10 00:00:00',
 '1992-03-23 00:00:00', '2018-10-15 00:00:00', '2017-09-01 00:00:00',
 '1959-04-01 00:00:00', '2018-10-31 00:00:00', '2017-12-17 00:00:00',
 '2018-09-12 00:00:00',
 ...
 '1996-04-03 00:00:00', '1990-12-05 00:00:00', '1973-04-21 00:00:00',
 '1994-03-18 00:00:00', '2004-06-17 00:00:00', '1980-05-24 00:00:00',
 '1983-01-31 00:00:00', '1986-04-15 00:00:00', '2002-04-04 00:00:00',
 '1995-01-17 00:00:00']
Length: 12154, dtype: datetime64[ns]

In [67]:
df['BILLDATE'].unique()

array(['2023-07-17 04:55:49 PM', '2023-07-07 03:37:16 PM', nan, ...,
       '2023-07-10 07:45:30 AM', '2023-07-10 08:24:05 PM',
       '2023-07-02 02:40:23 PM'], dtype=object)

In [68]:
df['DISCONNECTION_DT'].unique()

array([nan])

In [69]:
df = NonGOVT(df)

In [70]:
df = relGov(df)

In [71]:
df = faulty_Ageing(df)

In [72]:
df['BILLING_STATUS'].unique()

array(['Live', 'Account Closed', 'Temp.Disconnected', 'First Bill Issued',
       'Bill Stopped', 'Dismantled', 'New Connection'], dtype=object)

In [73]:
df = billable(df)


In [74]:
df['BILLING_STATUS'].unique()

array(['Live', 'Account Closed', 'Temp.Disconnected', 'First Bill Issued',
       'Bill Stopped', 'Dismantled', 'New Connection'], dtype=object)

In [75]:
df['AMOUNT'] = pd.to_numeric(df['AMOUNT'],errors="coerce")
df = arrear(df)

In [76]:
df['BILLING_STATUS'].unique()

array(['Live', 'Account Closed', 'Temp.Disconnected', 'First Bill Issued',
       'Bill Stopped', 'Dismantled', 'New Connection'], dtype=object)

In [77]:
# df['LAST_PAID_DATE'] = pd.to_datetime(df.LAST_PAID_DATE)
# df['SUPP_RELEASE_DATE'] = pd.to_datetime(df.SUPP_RELEASE_DATE)

In [78]:
df.dtypes


DIVISION         object
SCNO             object
ACCOUNTNO         int64
SUBDIVISION      object
SUBSTATION       object
                  ...  
gov_flag           bool
relGov           object
faulty_Ageing    object
bilable_flag       bool
ARREAR_FILTER    object
Length: 64, dtype: object

In [79]:
df=CatLoad(df)

In [80]:
# df1 = df[(df['gov_flag'] == False)]

In [81]:
# df1=date(df1)

In [82]:
# df1 = df1[df1['SUPP_RELEASE_DATE'] >= pd.Timestamp(2018, 4, 1)]

In [83]:
# df1 =  df1[df1['bilable_flag'] == True]

In [84]:
# df1 = df1[((df1['LAST_PAID_DATE'] < pd.Timestamp(2022, 12, 1)) | (pd.isnull(df['LAST_PAID_DATE'] )))]

In [85]:
# df1 =  df1[df1['BILLING_STATUS'] == 'Temp.Disconnected']

In [22]:
df_duplicate.sort_values(by=['NAME','FATHER_NAME','VILLAGE_NAME'], inplace=True,ascending=False)

C:\Users\pc\AppData\Local\Temp\ipykernel_3324\255396345.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_duplicate.sort_values(by=['NAME','FATHER_NAME','VILLAGE_NAME'], inplace=True,ascending=False)


In [34]:
df_duplicate.head()

,BOOK_CODE,DIVISION,DIVISION_ID,SUBDIVISION,SUBSTATION,FEEDER,SUPPLY_TYPE,SCNO,OLDSCNO,BILLDATE,...,ROUTE_NAME,PAYMENT_TYPE,PLOOM_CHG,PTW_REBATE,PTW50REBATE,WHATSAPP_NO,INOPERATIVE_AMT,INOPERATIVE_DATE,KYC_CREATE_DATE,HV_BILL_FLAG
7619,3615,EDD-I-BAHRAICH,49,EDSD I BAHRAICH,BAXIPURA_354921,RURAL_35492110453RUL,17,4913615110560,NaN,NaN,...,NaN,NaN,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
16523,3515,EDD-I-BAHRAICH,49,EDSD I BAHRAICH,BAXIPURA_354921,RURAL_35492110453RUL,17,4913515110151,NaN,2023-07-10 10:11:33 AM,...,BEATMVV08528,NaN,0,0.0,0.0,9.565258e+09,NaN,NaN,09-FEB-23,NaN
100482,5418,EDD-III-BAHRAICH-KAISERGANJ,KG,EDSDIIIBAHRAICHRANJEETPUR,CHIRAIYA TAND_354923,SHIVDAHA_35492140464SVA,17,KG25418118721,KG25418118721,2023-07-05 08:16:37 AM,...,BEATMVV09192,NaN,0,0.0,0.0,NaN,NaN,NaN,03-02-23,NaN
137250,5451,EDD-III-BAHRAICH-KAISERGANJ,KG,EDSDIIIBAHRAICHRANJEETPUR,CHIRAIYA TAND_354923,SHIVDAHA_35492140464SVA,17,KG15451118721,KG15451118721,2023-07-09 03:33:44 PM,...,BEATMVV09190,NaN,0,0.0,0.0,9.721458e+09,NaN,NaN,21-06-23,NaN
231560,414,EDD-II-BAHRAICH-NANPARA,NP,EDSD I NANPARA,SAHABA/BABAGANJ,NaN,17A,NP20414112756,NP10451112756,2023-07-10 08:34:45 AM,...,BEATMVV09384,NaN,0,0.0,0.0,9.918476e+09,NaN,NaN,27-06-23,NaN


In [87]:
# df1.head(10)

In [88]:
# df1=df1.groupby('DIVISION').head(10).reset_index(drop=True)

In [89]:
# df1.head()

In [23]:
start = time.time()
# df.to_csv("EDC_new.CSV")
# df.to_csv(f"{month}/{circle}/{circle}_jul.csv") 
df_duplicate.to_csv(f"{month}/{circle}/{circle}_jul_duplicate_saubhag.csv")
# df1.to_excel(f"{month}/top_10-td-nonrapdrp.xlsx")
end = time.time()
print("CSV:", end - start)

CSV: 4.10788106918335


In [91]:

# # import pandas
# import pandas as pd
   
# # read csv data
# df1 = pd.read_csv('Student_data.csv')
# df2 = pd.read_csv('Course_enrolled.csv')
   
# Left_join = pd.merge(df1, 
#                      df2, 
#                      on ='Name', 
#                      how ='left')
# Left_join

In [92]:
import pandas as pd
from datetime import datetime
from datetime import date

d = {'col0': [1, 2, 6], 
     'col1': [3, 8, 3], 
     'col2': ['17.02.1979', '11.11.1993', '01.08.1961']}

# df = pd.DataFrame(data=d)

# def calculate_age(born):
#     born = datetime.strptime(born, "%d.%m.%Y").date()
#     today = date.today()
#     return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

# df['age'] = df['col6'].apply(calculate_age)
# print(df)

In [93]:
df = pd.DataFrame(data=d)

In [94]:
def calculate_age(born):
    born = datetime.strptime(born, "%d.%m.%Y").date()
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))


In [95]:
df['age'] = df['col2'].apply(calculate_age)
print(df)

   col0  col1        col2  age
0     1     3  17.02.1979   44
1     2     8  11.11.1993   29
2     6     3  01.08.1961   62
